<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S12_intro_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q -U gtbook


# Robot Actions

> By executing actions, robots change the state of the world, as well as their own state.

<img src="Figures1/S12-Robot_menagerie-03.jpg" alt="Splash image with thinking robot" width="40%" align=center style="vertical-align:middle;margin:10px 0px">

Robots are of interest to us because they have the ability to move in
their environment, manipulate objects, and generally effect changes in
the world state. In order for a robot to plan actions that achieve its
goals, it must have a representation of its own actions, along with a
characterization of how those actions affect the state of the world. The
specific choice of representation depends on the task at hand, and the
level of abstraction at which the task is represented.

## Symbolic Task-Level Actions

For high-level, task planning, it is often sufficient to use a
high-level, symbolic representation scheme such as that described above for the
STRIPS style planners. In this case, the representation of an action
includes a set of conditions that must be true for the action to be
applicable, and a set of changes that will result if the action is
executed. 
Consider the example of Figure
<a href="#fig:blocks-world" data-reference-type="ref" data-reference="fig:blocks-world">1</a>.
We can define an action that causes the robot to grasp a block that is
resting on another block as follows:

**Action**: *UNSTACK(?X, ?Y)*  
**Preconditions**: On(?X, ?Y)  
**Delete**: On(?X, ?Y)  
**Add**: InGripper(?X), Free(?Y)

In this example, the symbols *?X* and *?Y* denote variables that are
instantiated based on which objects are the target of the desired
action. The *preconditions* provide a set of conditions that must hold
in the world state in order for this action to be valid. The *add* and
*delete* lists specify the changes to the world state that will occur
once the action is applied.
This style of action representation is popular for domains in which high-level descriptions
of problems and plans suffice,
and AI planning researchers have developed the
*Planning Domain Definition Language (PDDL)* 
as a standard language for expressing such problems
(though, by now, many variations for this standard already exist).


## Discrete Time Systems

As might be expected, for each of the kinds of state representation
described above, there is a corresponding action representation. For the
case of a robot that moves in a grid (as in Figure
<a href="#fig:chap1-occupancy-grid" data-reference-type="ref" data-reference="fig:chap1-occupancy-grid">3</a>),
we might denote the robot state at time $t$ as $x_t = (r_t,c_t)$, in
which $r,c$ denote a row and column index into the grid. We could
specify a unique action for each of the four possible directions of
motion in the grid, but it is more convenient in this case to define
actions to be a function of a control input, as $x_{t+1} = f(x_t, u_t)$,
in which $u_t$ denotes the control action that is applied at time $t$.
Here there are four possible choices for $u$, corresponding to moving
up, down, left, or right. For example, when moving to the left, we would
have

$$x_{t+1} = \left[\begin{array}{c}r_{t+1} \\ c_{t+1}\end{array}\right] 
 = f(x_t, \mathrm{left})
        = \left[\begin{array}{c}r_{t} \\ c_{t} - 1 \end{array}\right]$$

For systems described as $x_{t+1} = f(x_t, u_t)$, the control input $u$
provides the connection between planning and acting. Planning can be
characterized as the search for a set of control inputs
$u_1, u_2, \dots$ that will transform the world (including the robot
itself) from its initial state into the desired goal state. For
discrete-time systems that are represented using discrete states, this
can often be accomplished using various graph search algorithms,
such as the A* algorithm.

## Continuous Time Systems

For the case of continuous time robot systems that evolve on continuous
state spaces, we typically write the dynamic equations of motion as
$\dot{x}(t) = f(x,u)$ in which $x(t) = (q(t), \dot{q}(t))$ and
$\dot{x}(t) = (\dot{q}(t), \ddot{q}(t))$ and $u$ denotes a control
input. The system dynamics for different robots take a variety of
special forms. For example, a linear systems whose behavior does not
vary with time (so called *linear, time-invariant* or *LTI* systems) can
be written as 

$$\dot{x}(t) = Ax(t) + B u(t)$$

 This is the most basic
kind of continuous time dynamical system, yet such systems can often be
used to provide reasonably good approximations to even fairly complex
nonlinear systems.

Finding a suitable control input $u(t)$ to drive an arbitrary nonlinear
system to its goal state can be a very difficult problem. One way to
solve this problem is to discretize the system, and use numerical
optimization methods to find a discrete-time control law that will
achieve the goal. We discuss these issues, along with other specific
forms of system dynamics for a variety of robot systems in subsequent
chapters.

## Characteristics of Robot Actions

All of the above action representations share a few common characteristics.

First, actions have local effects, both spatially and temporally.
Actions are executed *here* and *now* and their effects are immediate.
This is perhaps most obvious for continuous time systems, in which actions affect the
time derivative of the system trajectory. 
This local nature of actions brings a significant computational benefit.
Planners need not worry about the entire state of the world over the entire time history
of execution. Rather, planners can focus on building plans one step at a time,
constructing a sequence of local changes that ultimately bring about the desired world
state.
Furthermore, this local property allows us to simplify the descriptions of
actions, including only those aspects of the world that spatially relevant.
Note, for example, that the precondition for picking up a block depends
only on what is immediately on top of the block, not on the arrangement of other
blocks that might be on the table.

A second characteristic of the actions described above is that the do not include any consideration
of uncertainty.
In real-world robotics problems, the robot rarely knows the exact state of the world,
and it cannot predict with certainty the effects of its own actions on the world.
To cope with such uncertainties, we often use probability theory to model the effects
of actions.
For example, if the action is to move one space to the left in a grid, we might model
the action as

$$ f(x_t, \mathrm{left})
        = \left[\begin{array}{c}r_{t} \\ c_{t} - 1 + w_t \end{array}\right]$$

in which $w_t$ represents an unknown disturbance.  If we can say something about
the probability distribution for $w_t$, for example that $w_t \in \{ -1, 0 , 1\}$ and
that all outcomes are equally probably, 
then
we can use probability theory to reason about which actions to apply to maximize
the probability of successful execution.
